# Reproducability and Seeding
This notebook showcases how TorchSig handles random seeding to allow reproducable experiments.

---

## Iterable Dataset
The TorchSigIterableDataset class inherits torch IterableDataset, and is used to sample synthetic datasets at runtime

In [ ]:
from torchsig.datasets.datasets import TorchSigIterableDataset
from torchsig.utils.defaults import TorchSigDefaults

dataset_metadata = TorchSigDefaults().default_dataset_metadata
print(dataset_metadata)

Creating the dataset without seeding; this print statement will create a different random signal every time you call it.

If you run this cell multiple times, or if you reload this notebook and run it again it will not produce the same signal.

In [ ]:
dataset = TorchSigIterableDataset(metadata=dataset_metadata)
print(next(dataset).data)
print(next(dataset).data)
print(next(dataset).data)

In [ ]:
dataset = TorchSigIterableDataset(metadata=dataset_metadata)
print(next(dataset).data)
print(next(dataset).data)
print(next(dataset).data)

## Seeding
All torchsig Transforms, Datasets, DatasetMetadata objects, and DataLoaders are seedable objects.

This means they all have a .seed(N) method, which can be called to input a random seed. If no seed is given, the seedable object with produce its own seed and generate different random numbers every time you run your code.

If you want reproducable experiments, you generally will want to call .seed(N) on some integer N of your choosing. This will ensure the same 'random' outcomes occur each time the code is executed.

You don't need to seperately seed connected objects. If a dataset contains several transforms, seeding the dataset is enough to also correctly seed all of its transforms.

In general, you will only need to call .seed() on the top level object you are using (typically either a dataset or a data loader).

NOTE: Calling numpy.random.seed will not seed torchsig datasets; they should always be seeded explicitely if a seed is desired

### Seeding the Dataset
Here the same dataset from above is seeded; this code will produce the same random signals every time it is run

In [ ]:
dataset = TorchSigIterableDataset(metadata=dataset_metadata)
dataset.seed(42)
print(next(dataset).data)
print(next(dataset).data)
print(next(dataset).data)

In [ ]:
dataset = TorchSigIterableDataset(metadata=dataset_metadata)
dataset.seed(42)
print(next(dataset).data)
print(next(dataset).data)
print(next(dataset).data)

### Seeding a DataLoader
On a single worker threads/process, seeding a dataset alone is sufficient to reproduce results correctly.

Since DataLoaders typically use several different worker threads/processes, each with a copy of the dataset, we generally want each worker to have a different seed for its copy of the dataset, so that its randomly generated data does not match that of the other workers.

This is unneccessary for loading static data from files, but it is needed for loading on-the-fly random generated data correctly.

To address this issue, TorchSig exposes a WorkerSeedingDataLoader, which will seed a torchsig dataset differently in all workers.

NOTE: WorkerSeedingDataLoader uses it's own worker init function, and is not compatible with other custom worker init functions; the exact data generated will still depend on the configuration of workers, so it will not produce the same data with different worker counts


In the code below, we create and seed a WorkerSeedingDataLoader for our dataset

In [ ]:
from torchsig.utils.data_loading import WorkerSeedingDataLoader

batch_size = 8
num_workers = 2

In [ ]:
dataset = TorchSigIterableDataset(metadata=dataset_metadata)
dataloader = WorkerSeedingDataLoader(
    dataset, batch_size=batch_size, collate_fn=lambda x: x
)
dataloader.seed(42)

data = [x.data for x in next(iter(dataloader))]
print(data)

Because we are seeding our dataloader with the same seed value, and because both dataloaders have the same worker count, this code will produce the same batch of signals every time it is run

In [ ]:
dataset = TorchSigIterableDataset(metadata=dataset_metadata)
dataloader = WorkerSeedingDataLoader(
    dataset, batch_size=batch_size, collate_fn=lambda x: x
)
dataloader.seed(42)

data = [x.data for x in next(iter(dataloader))]
print(data)

Tensors expect each array to have the same size. This presents a challenge when there is no signal present, ex: `num_signals_min == 0`, which is the default for `DatasetMetadata()`. All of the IQ time-series within a dataset will always be same, even when there is no signal present, because the underlying noise is created. However, when there is no signal the metadata becomes an empty list. Therefore when viewing the metadata in tensor format there will be zero-padded fields. When processing the output of the data loader the zero-padding will need to be undone.